![image](https://github.com/ecastillot/SeisMonitor/blob/master/docs/figures/seismonitor.PNG?raw=true)

**This code is necessary on colab to install SeisMonitor.** 

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install SeisMonitor
    !pip install git+https://github.com/ecastillot/EQTransformer.git@master
    !pip install git+https://github.com/wayneweiqiang/GaMMA.git

# Basic inputs

In [ ]:
import os
import pandas as pd
from SeisMonitor.monitor.picker.ai import EQTransformer,EQTransformerObj
from SeisMonitor.monitor.picker import utils as piut
from SeisMonitor.monitor.associator.ai import GaMMA,GaMMAObj
from SeisMonitor.monitor.associator import utils as asut
from SeisMonitor.monitor.locator.nlloc.nlloc import NLLoc
from SeisMonitor.monitor.locator import utils as lut
from obspy.core.utcdatetime import UTCDateTime
from obspy.clients.fdsn import Client as FDSNClient
from SeisMonitor.core.objects import WaveformRestrictions,Provider
from SeisMonitor.monitor.downloader.seismonitor import MseedDownloader

monitor_path = os.path.join(os.path.dirname(os.getcwd()),"6m")

sgc_rest = WaveformRestrictions(network="CM",
                    station="URMC,CLEJA,VILL,PRA,ORTC,GARC,FLO2,CHI,YOT",
                    location="*",
                    channel="*",
                    starttime=UTCDateTime("2019-12-24T19:00:00.000000Z"),
                    endtime=UTCDateTime("2019-12-24T23:00:00.000000Z"),
                    location_preferences=["","00","20","10","40"],
                    channel_preferences=["HH","BH","EH","HN","HL"],
                    filter_networks=[], 
                    filter_stations=[],
                    filter_domain= [-83.101,-64.549,-2.229,14.945],
                    )
sgc_client = FDSNClient('http://sismo.sgc.gov.co:8080')

gamma_region = [-76.729, -72.315,1.55, 5.314,0, 150]
nlloc_region = [-85, -68,0, 15,-5, 205]
nlloc_delta_in_km = 2.5

vel_model = {"depth":[0,4,23,32,40,100,200],
            "vp":[4.8,6.6,7,8,8.1,8.2,8.3],
            "rho":[2.7,2.7,2.7,2.7,2.7,2.7,2.7]}

vel_model = pd.DataFrame.from_dict(vel_model)

# Downloader

In [3]:

sgc_provider = Provider(sgc_client,sgc_rest)
md = MseedDownloader(providers=[sgc_provider])

stations_path = os.path.join(monitor_path,"stations")
downloads_path = os.path.join(monitor_path,"downloads")
mseed_storage = os.path.join(downloads_path,"{station}/{network}.{station}.{location}.{channel}__{starttime}__{endtime}.mseed")

inv,json = md.make_inv_and_json(stations_path)
md.download(mseed_storage,
            picker_args={"batch_size":100,
                        "overlap":0.3,"length":60},
            chunklength_in_sec=7200,n_processor=None)

# Picker

In [ ]:
from SeisMonitor.utils4examples import clone_seismonitor_data

models = os.path.join(os.path.dirname(os.getcwd()),"picking_models")
clone_seismonitor_data(models,branch="models")
eqt_model = os.path.join(models,"EQTransformer_models","EqT_model.h5")

print("Models dir: ",models)
print("Important folders in your models",os.listdir(models))

In [5]:
eqtobj = EQTransformerObj(model_path=eqt_model,
            n_processor = 6,
            overlap = 0.3,
            detection_threshold =0.1,
            P_threshold = 0.01,
            S_threshold = 0.01,
            batch_size = 100,
            number_of_plots = 0,
            plot_mode = 1 ) 

out_dir = os.path.join(monitor_path ,"picks","eqt")
result = os.path.join(monitor_path ,"picks","eqt","seismonitor_picks.csv")

eqt = EQTransformer(eqtobj)
eqt.pick(downloads_path,stations_path,out_dir)
piut.eqt_picks_2_seismonitor_fmt(out_dir,downloads_path,result)

# Associator

In [6]:

gc = GaMMAObj(gamma_region,"EPSG:3116",
                use_amplitude = False,
                use_dbscan=False,
                calculate_amp=False)

inv = os.path.join(stations_path,"inv.xml")
gamma_dir = os.path.join(monitor_path,"gamma_asso","eqt")

g = GaMMA(gc)
obspy_catalog, df_catalog,df_picks = g.associate(result,inv,gamma_dir)
print(obspy_catalog)

# Nlloc

installing nlloc for ubuntu machines.

In [ ]:
import platform
from SeisMonitor.monitor.locator.nlloc.utils import download_nlloc
if "linux" in platform.system().lower():
  test = ! lsb_release -a
  test = "__".join(test)
  if "ubuntu" in test.lower():
    ! sudo apt-get install -y cmake
    nlloc_path = "/opt/NLLoc"
    download_nlloc(nlloc_path)
    print("NLLoc now is working properly" )
  else:
    nlloc_path = None
    print("NLLoc was not installed using this script. Please install NonLinLoc from: https://github.com/alomax/NonLinLoc" )
else: 
    nlloc_path = None
    print("NLLoc was not installed using this script. Please install NonLinLoc from: https://github.com/alomax/NonLinLoc" )

In [ ]:
nlloc_out_dir = os.path.join(monitor_path,"nlloc")
inv = os.path.join(stations_path,"inv.xml")

vel_model = lut.VelModel(vel_model,model_name="myown_vel_model")
stations = lut.Stations(inv)

nlloc = NLLoc(
        core_path = nlloc_path, ### type your NLLoc path, 
        agency="SeisMonitor",
        region = nlloc_region,
        vel_model = vel_model,
        stations = stations,
        delta_in_km = nlloc_delta_in_km,
        tmp_folder=os.path.join(os.getcwd(),"NLLoc_grid") ### CHANGE PATH TO YOUR OWN PATH AND ALSO TAKE IN MIND THAT CONSUME DISK
        )

In [ ]:
nlloc.compute_travel_times()
print(nlloc.tmp_folder)
print("nlloc.tmp_folder dir: ",nlloc.tmp_folder)
print("Important folders in nlloc.tmp_folder directory",os.listdir(nlloc.tmp_folder))

In [ ]:
eqt_catalog = os.path.join(gamma_dir,"associations.xml")
eqt_nlloc_catalog = nlloc.locate(catalog=eqt_catalog,
                            nlloc_out_folder= nlloc_out_dir,
                            out_filename = "LOC.xml",
                            out_format="SC3ML" )
print(eqt_nlloc_catalog)